In [ ]:
import ast, tqdm, pandas as pd
from llm import call_llm

dev_set = pd.read_csv('./dev_set.csv')
dev_set

In [ ]:
dev_set['zero_shot'] = ''
for idx, row in tqdm.tqdm(dev_set.iterrows(), desc="Inferencing on Dev Set", total=dev_set.shape[0]):
    conversation = ast.literal_eval(row['conversations'])
    dev_set.iloc[idx]['zero_shot'] = call_llm(conversation[0]['value'])

In [ ]:
train_set = pd.read_csv('./train_set.csv')

def get_few_shot_examples(n):
    few_shot_examples = ''
    for _, row in train_set.sample(n=n).reset_index(drop=True).iterrows():
        conversation = ast.literal_eval(row['conversations'])

        few_shot_examples += f'Example: {_+1}\n\n'
        few_shot_examples += conversation[0]['value']
        few_shot_examples += f"OUTPUT: {conversation[1]['value']}\n\n"
    
    return few_shot_examples.strip()

In [ ]:
dev_set['few_shot'] = ''
for idx, row in tqdm.tqdm(dev_set.iterrows(), desc="Inferencing on Dev Set", total=dev_set.shape[0]):
    conversation = ast.literal_eval(row['conversations'])[0]['value']
    content = conversation + '\n\n' + "Here are few examples for your reference:\n\n" + get_few_shot_examples(2)
    dev_set.iloc[idx]['few_shot'] = call_llm(content)

In [ ]:
dev_set.to_csv('./dev_set_zero_few_shot.csv', index=False)